In [2]:
import numpy as np
from scipy import linalg
import time as time
import sys
import matplotlib.pyplot as plt

#Toeplitz for scrodinger one electron egn
def creatToeplitz_Matrix(ru_min,ru_max,n):
    h = (ru_max - ru_min)/n
    A= np.zeros((n,n), dtype=float)
    a = 2.0/h**2
    b = -1.0/h**2
    A[0,0]=a+((ru_min+(1*h)))
    A[0,1]=b
    A[n-1,n-1]=a
    A[n-1,n-2]=b
    for i in range(1,n-1):
        A[i,i]=a+(((ru_min+(i+1)*h))*((ru_min+(i+1)*h)))
        A[i,i+1] = b
        A[i,i-1]=b
    return A
    print("A")
    print (A)

# calculate maximum off diagnal value to cal  rotational nagle to make offdia zer0
def maxoffdia(A,n):
    maximum = 0
    for i in range(0,n):
        for j in range(i+1,n):
            if(np.abs(A[i][j]) > maximum):
                maximum = np.abs(A[i][j])
                k = i
                l = j
                
    return maximum,l,k

# jacobi rotation method

def rotate(A,R,k,l,n):
    if (A[k][l] != 0):
        tau = (A[l][l] - A[k][k])/(2*A[k][l])
            # to get smallest abs value of rot angle 
        if (tau >= 0):
            t=1/(tau+np.sqrt(1+tau*tau))
        else:
            t=-1/(-tau+np.sqrt(1+tau*tau))
        c = 1/np.sqrt(1+t**2)
        s = c*t
    else:
        # means a[k][l] is zero
            c = 1.0
            s = 0.0
   
    
    # calculating other elements in the array affected bythe rotation 
    for i in range(0,n):
            if i!=k and i!=l:
                a_ik = A[i][k]
                a_il = A[i][l]
                A[i][k] = c*a_ik - s*a_il
                A[k][i] = A[i][k]
                A[i][l] = c*a_il + s*a_ik
                A[l][i] = A[i][l]
            r_ik = R[i][k]
            r_il = R[i][l]
            R[i][k] = c*r_ik - s*r_il
            R[i][l] = c*r_il +s*r_ik
 # pre calculation of rotations manually defined by k and l 
    a_kk = A[k][k]
    a_ll = A[l][l]
    A[k][k] = c*c*a_kk - 2.0*c*s*A[k][l] + s*s*a_ll
    A[l][l] = s*s*a_kk + 2.0*c*s*A[k][l] + c*c*a_ll
    A[k][l] = 0.0
    A[l][k] = 0.0
            
##################       
# initialise array 
def jacobi_method(A, R, n):
    for i in range(0,n):
        for j in range(0,n):
            if i==j:
                R[i][j] = 1.0
            else:
                R[i][j] = 0.0
    # max error tolenrance to stop the algorithm 
    tol = 1.0e-10
    # max number of iterations 
    pow_n= 6
    max_iteration = n**pow_n
    iteration = 0
    max_offdiagonal,l,k = maxoffdia(A,n)
    
    start_time_jacobi = time.time()
    
    while abs(max_offdiagonal) > tol and iteration < max_iteration :
        max_offdiagonal,l,k = maxoffdia(A,n)
        # print(max_offdiagonal,l,k)
        rotate(A,R,k,l,n)
        iteration +=1
        
    #print(iteration)
       # print('number of iteration: ' ,iteration , '\n')
    end_time_jacobi=time.time()
    # calculting time for jacobi rotation method 
    exe_time_jacobi=end_time_jacobi-start_time_jacobi
   
   # print(A)
    # eigen array and sorting it 
    eigen=np.zeros((n), dtype=float)
    for i in range(0,n):
        eigen[i]=A[i][i]
    eigen=np.sort(eigen)
    print()
    print()
    print("calculated eigen values from jacobi method")
    print(eigen)
    
    # calculating time for built in linalg function 
    start_time_builtin_linalg_fn=time.time()
    w,v = linalg.eig(A)
    w=np.sort(w)
    v=np.sort(v)
    #print(w)
    #print(v)
    end_time_builtin_linalg_fn=time.time()
    exe_time_builtin_linalg_fn= end_time_builtin_linalg_fn-start_time_builtin_linalg_fn
    
    print()
    print()
    print("functional eigen values from numpy")
    print (w)
   # print("eigen vectors")
   # print(v)
   # print()
   # print()
    #plt.figure()
   # x=np.linspace(0,10,50)
    #plt.plot(x,v[:,1])
    #plt.show()
   # print("execution time jacobi")
    #print(exe_time_jacobi)
   # print("execution time builtin line alg")
    #print(exe_time_builtin_linalg_fn)
    return exe_time_jacobi,exe_time_builtin_linalg_fn,iteration,eigen[0], eigen[1],eigen[2] 
 #   print(max_iteration)
    
    #########
  # rotation matrix   
def Creat_rotate_matrix(n):
    R = np.zeros((n,n), dtype=float)
    
    return R
#####################

#writing time values to file 
outf = open('out_one electron.txt', 'w+')
outf.write("n \t Rho_max\t exe time_jacobi\t exe_time_linealg fn\t no.of transformations\t first three eigen values\n")
outf.close()
for i in range(50 , 100 , 50):
    n =i
    rho_max=10
    R = Creat_rotate_matrix(n)
    A = creatToeplitz_Matrix(0,rho_max,n) 
                #print (n)
    print()
    print()
                #print (R)
                #max_offdiagonal,l,k = maxoff(A,n)
                  #  print(max_offdiagonal,l,k)
    J,F,I, E0, E1,E2=jacobi_method(A, R, n)
    
    outf=open('out_one electron.txt', 'a')
    outf.write(str(n))
    outf.write("\t")
    outf.write(str(rho_max))
    outf.write("\t")
    outf.write("{:.6f}".format(J))
    
    outf.write("\t")
    outf.write("{:.6f}".format(F))
    outf.write("\t")
    outf.write("{:.0f}".format(I))
    outf.write("\t")
    outf.write("{:.6f}".format(E0))
    outf.write("\t")
    outf.write("{:.6f}".format(E1))
    outf.write("\t")
    outf.write("{:.6f}".format(E2))
    outf.write("\n")
    #print(i)
    outf.close()














calculated eigen values from jacobi method
[  2.99024563   6.94098363  10.85020118  14.71741602  18.54198962
  22.32320554  26.06028602  29.75239466  33.39863425  36.99804201
  40.54958309  43.47191316  44.05214237  47.50451446  50.90539164
  54.25334916  57.54682745  60.78411012  63.96329688  67.08227029
  70.1386576   73.12979642  76.05273392  78.90432294  81.68148711
  84.3815931   87.00247333  89.54120059  91.99091227  94.33587957
  96.54540149  98.57708313 100.46909521 102.45534797 104.70415343
 107.22220821 109.9949941  113.02041081 116.30556428 119.86449757
 123.71784096 127.89363617 132.42924696 137.37473755 142.79870449
 148.79879069 155.52229382 163.21211883 172.33257848 184.0754418 ]


functional eigen values from numpy
[  2.99024563+0.j   6.94098363+0.j  10.85020118+0.j  14.71741602+0.j
  18.54198962+0.j  22.32320554+0.j  26.06028602+0.j  29.75239466+0.j
  33.39863425+0.j  36.99804201+0.j  40.54958309+0.j  43.47191316+0.j
  44.05214237+0.j  47.50451446+0.j  50.90539164+